In [22]:
import yaml
import itertools
import numpy as np
import pylab as plt
from math import *
from types import *
from operator import itemgetter
from scipy.special import hyp2f1
from inspect import getargspec
from iminuit import Minuit,util
%matplotlib inline

In [43]:
class ZhaoProfiles(object):
    """ Class defining the mass and density of the Zhao profile """
    
    def __init__(self, *params):
        self.params = params
    
    def density(self,x):
        a, b, c = self.params
        return 1. / x**c / (1.+x**a)**((b-c) / a)

    def mass(self, x):
        a, b, c = self.params
        return x**(3.-a) * hyp2f1( (3-a)/b, (c-a)/b, (b-a+3.)/b, -x**b )

In [44]:
class DMprofile(object):
    """ Class returning the DM mass and density functions,
        depending on the profile chosen """

    def __init__(self, profiles):
        if 'core' in profiles:
            self.profile = ZhaoProfiles(1.,3.,0.)

    def DM_density(self, x):
        return self.profile.density(x) 

    def DM_mass(self,x):
        return self.profile.mass(x)

In [44]:
class SigmaLOS(object):
    def __init__(self, *funcs):
        self.funcs = funcs
    
    def get_signature(self):
        func_args = []
        for func in self.funcs:
            func_args.append(self.get_args_func(func))
        return list(itertools.chain(*func_args))
    
    def get_args_func(self,func):
        signature = getargspec(func)[0]
        if signature[0] == 'x':
            return signature[1:]
        else:
            raise ValueError, 'first argument should be "x"'
    
    def __call__(self, *freepars):
        result = np.empty_like(self.funcs)
        l = 0 
        for i,func in enumerate(self.funcs):
            num_args = len(self.get_args_func(func))
            func_vals = freepars[l:num_args+l]
            l += num_args
            result[i] = func(self.data[0],*func_vals)
        
        return np.power(result.prod() - self.data[1],2).sum()

In [ ]:
class loglike(object):    
    def __init__(self, data, sigma, like='Gauss'):
        self.R = data[0]
        self.v = data[1]
        self.dv = data[2]
        self.sigma = sigma
        self.like = like
    
    def GaussLike(self, *params):
        term1 = 0.5*np.power(self.v-self.v.mean(),2) / (self.sigma(*params))
        term2 = np.log(self.sigma(*params) + self.dv**2)
        return (term1 + term2).sum()
    
    def __call__(self, *freepars):
        if self.like=='Gauss':
            return self.GaussLike(*freepars)

In [45]:
LL = loglike((x_arr, y_arr),h,f)
LL.get_signature()

['c', 'd', 'a']

In [46]:
class Fitter(object):
    def __init__(self, loglike):
        self.loglike = loglike
        global_loglike = loglike
        self.settings = {'errordef':0.5,'print_level':1,'pedantic':False}
        
    def sync(self):
        for key in self.loglike.get_signature():
            self.settings[key] = 1.
            self.settings['error_%s'%key] = 0.01
        return self.settings
    
    def MinuitFit(self, **kwargs):
        settings = self.sync()
        freepars = self.loglike.get_signature()
        strargs = ", ".join(freepars)
        fit_func = eval("lambda {args} : global_loglike({args})".format(args=strargs))
        mm = Minuit(fit_func, **settings)
        if 'tol' in kwargs: 
            mm.tol = kwargs['tol']
        fitresult = mm.migrad()
        return fitresult

In [47]:
global global_loglike
global_loglike = LL

m = MinuitFitter(LL)
m.fit()

FCN = 1.71600742665e-06 
 TOTAL NCALL = 87 
 NCALLS = 87 
 
 
 EDM = 1.71572751717e-06 
 GOAL EDM = 5e-06 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+ 
 Name 
 Value 
 Parab Error 
 Minos Error- 
 Minos Error+ 
 Limit- 
 Limit+ 
 FIXED 
 
 
 
 1 
 c 
 3.000023e+00 
 1.698859e-02 
 0.000000e+00 
 0.000000e+00 
 
 
 
 
 
 
 2 
 d 
 1.999890e+00 
 1.459450e-01 
 0.000000e+00 
 0.000000e+00 
 
 
 
 
 
 
 3 
 a 
 1.999239e+00 
 8.286725e-01 
 0.000000e+00 
 0.000000e+00 
 
 
 
 
 
 
 
 
 \begin{tabular}{|c|r|r|r|r|r|r|r|c|}
\hline
 & Name & Value & Para Error & Error+ & Error- & Limit+ & Limit- & FIXED\\
\hline
1 & c & 3.000e+00 & 1.699e-02 & & & & & \\
\hline
2 & d & 2.000e+00 & 1.459e-01 & & & & & \\
\hline
3 & a & 1.999e+00 & 8.287e-01 & & & & & \\
\hline
\end{tabular}

({'hesse_failed': False, 'has_reached_call_limit': False, 'has_accurate_covar': True, 'has_posdef_covar': True, 'up': 0.5, 'edm': 1.7157275171655186e-06, 'is_valid': True, 'is_above_max_edm': False, 'has_covariance': True, 'has_made_posdef_covar': False, 'has_valid_parameters': True, 'fval': 1.716007426647748e-06, 'nfcn': 87},
 [{'is_const': False, 'name': 'c', 'has_limits': False, 'value': 3.0000228125505197, 'number': 0, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 'error': 0.016988587942543723, 'is_fixed': False},
  {'is_const': False, 'name': 'd', 'has_limits': False, 'value': 1.9998896430788096, 'number': 1, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 'error': 0.14594503714370397, 'is_fixed': False},
  {'is_const': False, 'name': 'a', 'has_limits': False, 'value': 1.9992392498110232, 'number': 2, 'has_lower_limit': False, 'upper_limit': 0.0, 'lower_limit': 0.0, 'has_upper_limit': False, 